In [1]:
# adds the project specific libs to the sys path
import sys
from pathlib import Path

proj_lib_path = str(Path.cwd().parent / 'libs')
if proj_lib_path not in sys.path:
    sys.path.insert(1, proj_lib_path)
    print(f'added {proj_lib_path} to the sys.path')

added /Users/n0c09jf/code/github/tryml/projects/mlops-model-train/libs to the sys.path


In [2]:
from imagenet_classify_trainer.config import MODEL_DIR, DATA_DIR
import os

print(f'models area at {MODEL_DIR}')
print(f'data is at {DATA_DIR}')

models area at /Users/n0c09jf/code/github/tryml/projects/mlops-model-train/_models
data is at /Users/n0c09jf/code/github/tryml/projects/mlops-model-train/_data


In [3]:
from torchvision import models
from torch import nn
import torch

class TinyImageNet(nn.Module):
    def __init__(self, num_classes=10) -> None:
        super(TinyImageNet, self).__init__()
        self.backbone = models.resnet152()
        self.backbone._modules['fc'] = nn.Linear(in_features=2048, out_features=10, bias=True)
    
    def forward(self, X):
        return self.backbone(X)
    
    def predict(self, X):
        logits = self.forward(X)
        return torch.argmax(logits, 1)

model = TinyImageNet(num_classes=10)
model.eval()
model

/usr/local/Caskroom/miniconda/base/envs/mlops-model-train_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TinyImageNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
    

In [4]:
from imagenet_classify_trainer.pipeline import DataParallelTrainPipeline

pipe = DataParallelTrainPipeline()
pipe.run(train_epochs=1)

cuda available=False cuda device_count=0 cuda_devices=0 train_device=None


100%|██████████| 1/1 [00:31<00:00, 31.84s/it]


last epoch's -> {'train': {'loss': 0.0, 'score': 1.0}, 'val': {'loss': 53.521050453186035, 'score': 0.10748922413793104}}


100%|██████████| 4/4 [00:04<00:00,  1.18s/it]


test accuracy=0.09186422413793104
max memory allocated = []


In [12]:
import torch.distributed as dist
import os

os.environ["RANK"] = str(1)
os.environ["WORLD_SIZE"] = str(2)
os.environ["MASTER_ADDR"] = 'localhost'
os.environ["MASTER_PORT"] = '12355'
    
dist.init_process_group(backend=dist.Backend.GLOO)
dist.get_world_size()

KeyboardInterrupt: 